# Data Acquistion

The first phase of the project is to acquire all of the data that is needed for this project. The initial data required can be broken down into three separate data sets:

1. The FourSquare Top 30 Venues to Visit in Chicago
1. For each of the Top Site get a list of up restaurants in the surrounding area
1. The Chicago Police Department Crime Data for the last Year

---

# Import Libraries

In this section we import the libraries that will be required to process the data.

The first library is **Pandas**.  
Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas will be used to house each of the data sets.

The second library is **Requests**.
Requests is a Python HTTP library, released under the Apache2 License. The goal of the project is to make HTTP requests simpler and more human-friendly.

The next library in **BeautifulSoup**
Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [1]:
# Import Pandas to provide DataFrame support
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Requests
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

In [2]:
# Yaml is used to store some of the required configurations
import yaml

with open("./capstone_data/config.yaml", "r") as f:
    cfg = yaml.load(f)
 
search_params = {
    'client_id': cfg['client_id'],
    'client_secret': cfg['client_secret'],
    'intent': 'browse',
    'limit': 50,
    'v': cfg['version']
}

/home/vananh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


---

## FourSquare Top 30 Venues to Visit in Chicago

FourSquare does not actually provide an API that will return a list of the top venues to visit in a city. To get this list we can though use the FourSquare website directly to request the top sites in Chicago and then use BeautifulSoup to scrape the data we need. Once we have this starting data the other supplemental data we need to complete this dataset can be retrieved from using the FourSquare Venue API.

In [3]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get(
    "https://foursquare.com/explore?mode=url&near=Chicago%2C%20IL%2C%20United%20States&nearGeoId=72057594042815334&q=Top%20Picks")

# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(page.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')

A sample venue is is extracted from the `top_venues` list, formatted and shown below:

~~~~
<div class="venueDetails">
  <div class="venueName">
    <h2>
  <a href="/v/millennium-park/42b75880f964a52090251fe3" target="_blank">Millennium Park
  </a>
</h2>
  </div>
  <div class="venueMeta">
    <div class="venueScore positive" 
         style="background-color: #00B551;" 
         title="9.7/10 - People like this place">9.7</div>
    <div class="venueAddressData">
      <div class="venueAddress">201 E Randolph St (btwn Columbus Dr &amp; Michigan Ave), Chicago</div>
      <div class="venueData">
        <span class="venueDataItem">
          <span class="categoryName">Park</span>
          <span class="delim"> • </span>
        </span>
      </div>
    </div>
  </div>
</div>
~~~~

From this HTML the following data will be extracted:

* Venue Name
* Venue Score
* Venue Category 
* Venue HREF
* Venue ID (Extracted from the HREF)


### Create Top Venues Dataframe

The `top_venues` list, a sample of which is shown above, only contains some of the data required. In addition to the attributes extracted directly from the HTML code the following attributes are also required:
* Venue Address
* Venue Postalcode
* Venue City
* Venue Latitude
* Venue Longitude

These attributes will be obtained directly from FourSquare using the `venues` API. The process is as follows:
1. Create a new empty Pandas dataframe to hold the data for the Top Sites / Venues
1. Extract the available attributes from the HTML code
1. For each venue
    1. Contruct a URL to interagate the FourSquare Venue API for each top site
    1. Using the `venues` API and the URL request the data from FourSquare
    1. Get the properly formatted address and the latitude and longitude data from the returned JSON
    1. Write the data for each venue to the top venues dataframs

In [4]:
# The column names for the top venues dataframe
venue_columns = ['id', 
                 'score', 
                 'category', 
                 'name', 
                 'address',
                 'postalcode',
                 'city',
                 'href', 
                 'latitude', 
                 'longitude']

# Create the empty top venues dataframe
df_top_venues = pd.DataFrame(columns=venue_columns)

# For each venue in the BeautifulSoup HTML object
for venue in top_venues:
    
    # Extract the available attributes
    venue_name = venue.find(target="_blank").get_text()
    venue_score = venue.find(class_="venueScore positive").get_text()
    venue_cat = venue.find(class_="categoryName").get_text()
    venue_href = venue.find(class_="venueName").h2.a['href']
    venue_id = venue_href.split('/')[-1]

    if 'promotedTipId' in venue_id: 
        continue
        
    # Contruct the FourSquare venue API URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id, 
        cfg['client_id'],
        cfg['client_secret'],
        cfg['version'])
    
    # Request the venue data
    result = requests.get(url).json()
        
    # Get the properly formatted address and the latitude and longitude
    venue_address = result['response']['venue']['location']['address']
    venue_postalcode = result['response']['venue']['location']['postalCode']
    venue_city = result['response']['venue']['location']['city']
    venue_latitude = result['response']['venue']['location']['lat']
    venue_longitude = result['response']['venue']['location']['lng']
    
    # Add the venue to the top venues dataframe
    df_top_venues = df_top_venues.append({'id': venue_id,
                                          'score': venue_score,
                                          'category': venue_cat,
                                          'name': venue_name,
                                          'address': venue_address,
                                          'postalcode': venue_postalcode,
                                          'city': venue_city,
                                          'href': venue_href,
                                          'latitude': venue_latitude,
                                          'longitude': venue_longitude}, ignore_index=True)


In [5]:
# Verify the shape of the top venues dataframe
df_top_venues.shape

(30, 10)

In [6]:
# Verify the dtypes of the top venues dataframe
df_top_venues.dtypes

id             object
score          object
category       object
name           object
address        object
postalcode     object
city           object
href           object
latitude      float64
longitude     float64
dtype: object

In [7]:
# The score type needs to be converted to float
df_top_venues['score'] = pd.to_numeric(df_top_venues['score'], errors='coerce').fillna(0)

# Describe the score to see if there is nuch variance in the values
df_top_venues.score.describe()

count    30.000000
mean      9.440000
std       0.067466
min       9.300000
25%       9.400000
50%       9.400000
75%       9.500000
max       9.600000
Name: score, dtype: float64

In [8]:
# Review the head of the dataframe to make sure it looks as expected
df_top_venues.head()

,id,score,category,name,address,postalcode,city,href,latitude,longitude
0,42b75880f964a52090251fe3,9.6,Park,Millennium Park,201 E Randolph St,60601,Chicago,/v/millennium-park/42b75880f964a52090251fe3,41.882598,-87.624126
1,4b9d15c5f964a520478e36e3,9.5,Waterfront,Chicago Riverwalk,Chicago River,60601,Chicago,/v/chicago-riverwalk/4b9d15c5f964a520478e36e3,41.887280,-87.627217
2,4b06c066f964a52097ef22e3,9.5,Liquor Store,Binny's Beverage Depot,1720 N. Marcey St.,60614,Chicago,/v/binnys-beverage-depot/4b06c066f964a52097ef22e3,41.913076,-87.655170
3,4f2a0d0ae4b0837d0c4c2bc3,9.5,Deli / Bodega,Publican Quality Meats,825 W Fulton Market,60607,Chicago,/v/publican-quality-meats/4f2a0d0ae4b0837d0c4c...,41.886642,-87.648718
4,4c23e840a852c928594ae26c,9.5,Trail,Nature Boardwalk,2001 N Clark St,60614,Chicago,/v/nature-boardwalk/4c23e840a852c928594ae26c,41.918102,-87.633283


In [41]:
# Finally write the dataframe to a pickle file for restoring later
df_top_venues.to_pickle('./anh/capstone_pickles/top_venues.pkl')

---

## FourSquare Restaurent Recommendations Data

Using the the list of all venue id values in the Top Sites DataFrame and the FourSquare categoryID that represents all food venues, we now search for restaurants within a 500 meter radius.



The requests returns a JSON object which can then be queried for the restaurant details required. A sample restaurnt from the results returned is shown below:

~~~~
{  
    "referralId":"v-1538424503",
    "hasPerk":"False",
    "venuePage":{  
        "id":"135548807"
    },
    "id":"55669b9b498ee34e5249ea61",
    "location":{  
        "labeledLatLngs":[  
            {  
                "label":"display",
                "lng":-87.62460021795313,
                "lat":41.88169538551873
            }
        ],
        "crossStreet":"btwn E Madison & E Monroe St",
        "postalCode":"60603",
        "formattedAddress":[  
            "12 S Michigan Ave (btwn E Madison & E Monroe St)",
            "Chicago, IL 60603",
            "United States"
        ],
        "distance":155,
        "city":"Chicago",
        "lng":-87.62460021795313,
        "neighborhood":"The Loop",
        "cc":"US",
        "state":"IL",
        "address":"12 S Michigan Ave",
        "lat":41.88169538551873,
        "country":"United States"
    },
    "name":"Cindy's",
    "categories":[  
        {  
            "pluralName":"Gastropubs",
            "id":"4bf58dd8d48988d155941735",
            "name":"Gastropub",
            "primary":"True",
            "icon":{  
                "prefix":"https://ss3.4sqi.net/img/categories_v2/food/gastropub_",
                "suffix":".png"
            },
            "shortName":"Gastropub"
        }
    ]
}
~~~~

From this JSON the following attributes are extraced and added to the Dataframe:

* Restaurant ID
* Restaurant Category Name
* Restaurant Category ID
* Restaurant Nest_name
* Restaurant Address
* Restaurant Postalcode
* Restaurant City
* Restaurant Latitude
* Restaurant Longitude
* Venue Name
* Venue Latitude
* Venue Longitude

The only piece of data that is missing is the Score or Rating of the Restaurant. To get this we need to make another FourSquare API query using the id of the Restaurant.

Using just the data in this DataFrame we will be able to generate maps displaying the chosen Top List Venue and the best scored surrounding restaurants.

In [10]:
# The column names for the restaurants dataframe
restaurants_columns = ['id',
                       'score', 
                       'category', 
                       'categoryID', 
                       'name', 
                       'address',
                       'postalcode',
                       'city',
                       'latitude',
                       'longitude', 
                       'venue_name', 
                       'venue_latitude',
                       'venue_longitude']

# Create the empty top venues dataframe
df_restaurant = pd.DataFrame(columns=restaurants_columns)

# Create a list of all the top venue latitude and longitude
top_venue_lats = df_top_venues['latitude'].values
top_venue_lngs = df_top_venues['longitude'].values

# Create a list of all the top venue names
top_venue_names = df_top_venues['name'].values

# Iterate over each of the top venues
# The venue name, latitude and longitude are passed to the loop
for ven_name, ven_lat, ven_long in zip(top_venue_names, top_venue_lats, top_venue_lngs):
    
    # Configure additional Search parameters
    # This is the FourSquare Category Id for all food venues
    categoryId = '4d4b7105d754a06374d81259'
    radius = 500
    limit = 50
    
    # Contruct the FourSquare search API URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
        cfg['client_id'],
        cfg['client_secret'],
        ven_lat,
        ven_long,
        cfg['version'],
        categoryId,
        radius,
        limit)
    
    # Make the search request
    results = requests.get(url).json()
    
    # Want a good selection of Restaurents
    # If less than 10 are returned ignore
    if len(results['response']['venues']) < 10:
        continue
        
    # Populate the new dataframe with the list of restaurants
    # Get the values for each Restaurant from the JSON
    for restaurant in results['response']['venues']:
 
        # Sometimes the Venue JSON is missing data. If so ignore and continue
        try:
            # Get location details
            rest_id = restaurant['id']
            rest_category = restaurant['categories'][0]['pluralName']
            rest_categoryID = restaurant['categories'][0]['id']
            rest_name = restaurant['name']
            rest_address = restaurant['location']['address']
            rest_postalcode = restaurant['location']['postalCode']
            rest_city = restaurant['location']['city']
            rest_latitude = restaurant['location']['lat']
            rest_longitude = restaurant['location']['lng']
            
            # Contruct the FourSquare venue API URL to get the venues rating / score
            rest_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
                rest_id, 
                cfg['client_id'],
                cfg['client_secret'],
                cfg['version'])

            # Get the restaurant score and href
            result = requests.get(rest_url).json()
            rest_score = result['response']['venue']['rating']
            
            # Add the restaurant details to the dataframe
            df_restaurant = df_restaurant.append({'id': rest_id,
                                                  'score': rest_score,
                                                  'category': rest_category,
                                                  'categoryID': rest_categoryID,
                                                  'name': rest_name,
                                                  'address': rest_address,
                                                  'postalcode': rest_postalcode,
                                                  'city': rest_city,
                                                  'latitude': rest_latitude,
                                                  'longitude': rest_longitude,
                                                  'venue_name': ven_name,
                                                  'venue_latitude': ven_lat,
                                                  'venue_longitude': ven_long}, ignore_index=True)
            
        # If there are any issue with a restaurant ignore and continue
        except:
            continue



In [11]:
# Verify the shape of the restaurants dataframe
df_restaurant.shape

(406, 13)

In [12]:
# Verify the dtypes of the restaurants dataframe
df_restaurant.dtypes

id                  object
score              float64
category            object
categoryID          object
name                object
address             object
postalcode          object
city                object
latitude           float64
longitude          float64
venue_name          object
venue_latitude     float64
venue_longitude    float64
dtype: object

In [13]:
# Review the head of the dataframe to make sure it looks as expected
df_restaurant.head()

,id,score,category,categoryID,name,address,postalcode,city,latitude,longitude,venue_name,venue_latitude,venue_longitude
0,551192b1498e8695352d33e6,8.2,American Restaurants,4bf58dd8d48988d14e941735,Remington's,20 N Michigan Ave,60602,Chicago,41.882628,-87.624608,Millennium Park,41.882598,-87.624126
1,4e73881c922e0a374024c7b8,7.5,Coffee Shops,4bf58dd8d48988d1e0931735,Starbucks,8 N Michigan Ave,60602,Chicago,41.882478,-87.624701,Millennium Park,41.882598,-87.624126
2,49e749fbf964a52086641fe3,8.9,Gastropubs,4bf58dd8d48988d155941735,The Gage,24 S Michigan Ave,60603,Chicago,41.881202,-87.624481,Millennium Park,41.882598,-87.624126
3,4ed3dd7e93ad987b5112d29f,8.1,Bakeries,4bf58dd8d48988d16a941735,Panera Bread,2 N Michigan Ave,60602,Chicago,41.882228,-87.624703,Millennium Park,41.882598,-87.624126
4,4a63b5c3f964a5209fc51fe3,7.8,Pizza Places,4bf58dd8d48988d1ca941735,Pizano's Pizza,61 E Madison St.,60603,Chicago,41.882031,-87.625642,Millennium Park,41.882598,-87.624126


In [14]:
# Describe the score to see if there is nuch variance in the values
df_restaurant.score.describe()

count    406.000000
mean       7.843596
std        0.910268
min        5.200000
25%        7.300000
50%        8.000000
75%        8.600000
max        9.500000
Name: score, dtype: float64

In [15]:
# How many of the top 30 sites / venues had > 10 restaurants nearby
df_restaurant.venue_name.nunique()

11

In [16]:
# How many unique restaurant categories are there
df_restaurant.category.nunique()

71

In [17]:
# How many unique restaurants are there
df_restaurant.name.nunique()

238

In [18]:
# What arethe top 10 most frequently occuring restaurant types
df_restaurant.groupby('category')['name'].count().sort_values(ascending=False)[:10]

category
Coffee Shops                54
Pizza Places                27
Donut Shops                 26
Italian Restaurants         20
Sandwich Places             18
American Restaurants        16
New American Restaurants    15
Bakeries                    14
Mexican Restaurants         12
Cafés                       12
Name: name, dtype: int64

In [19]:
# Which restaurants have to highest average score
df_restaurant.groupby('category')['score'].mean().sort_values(ascending=False)[:10]

category
Delis / Bodegas                     9.500000
Chocolate Shops                     9.400000
Southern / Soul Food Restaurants    8.900000
Molecular Gastronomy Restaurants    8.900000
Breweries                           8.833333
Music Venues                        8.800000
Gourmet Shops                       8.800000
Scandinavian Restaurants            8.800000
Brazilian Restaurants               8.800000
Salad Places                        8.780000
Name: score, dtype: float64

In [78]:
# Finally write the dataframe to a pickle file for restoring later
df_restaurant.to_pickle('./anh/capstone_pickles/restaurants.pkl')

---

# Import and process the Chicago Crime DataSet
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago in the last year, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified.

| Column Name   | Type          | Description                                            | 
| :------------ | :------------ | :----------------------------------------------------- | 
| CASE#         | Plain Text    | The Chicago Police Department RD Number (Records Division Number), which is unique to the incident. | 
| DATE OF OCCURRENCE | Date & Time   | Date when the incident occurred. this is sometimes a best estimate. |
| BLOCK	        | Plain Text    | The partially redacted address where the incident occurred, placing it on the same block as the actual address. |
| IUCR	        | Plain Text    | The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e. |
| PRIMARY DESCRIPTION   | Plain Text    | The primary description of the IUCR code. |
| SECONDARY DESCRIPTION	| Plain Text    | The secondary description of the IUCR code, a subcategory of the primary description. |
| LOCATION DESCRIPTION | Plain Text | Description of the location where the incident occurred. |
| ARREST        | Plain Text    | Indicates whether an arrest was made. |
| DOMESTIC      | Plain Text    | Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act. |
| BEAT          | Plain Text    | Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74. |
| WARD	        | Number        | The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76. |
| FBI CD        | Plain Text    | Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html. |
| X COORDINATE	| Plain Text    | The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Y COORDINATE	| Plain Text    | The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LATITUDE	    | Number        | The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LONGITUDE	    | Number        | The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LOCATION	    | Location      | The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block. |
	




### Import the 2020 DataSet

The full dataset, 2001 to 2020. This makes processing the dataset difficult and time consuming. Only the 2020 data, 232153 records, will be used. In addition, the following clean-up steps are required:
1. Not all of the columns are required. The following columns are removed:
    1. IUCR
    1. ARREST
    1. DOMESTIC
    1. BEAT
    1. WARD
    1. FBI CD
    1. X COORDINATE
    1. Y COORDINATE
    1. LOCATION

In [39]:
# These are the columns that we want to keep.
# Columns not listed here won't be imported, speeding things up.
crime_keep_columns = ['CASE#',
                      'DATE  OF OCCURRENCE',
                      'BLOCK', 
                      ' PRIMARY DESCRIPTION',
                      'WARD',
                      'LATITUDE',
                      'LONGITUDE']

In [59]:
# Download csv
!wget -O './anh/capstone_data/crimes.csv' https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv?accessType=DOWNLOAD

--2020-07-13 11:41:59--  https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./anh/capstone_data/crimes.csv’

./anh/capstone_data     [          <=>       ]  41,28M   322KB/s    in 2m 1s   

2020-07-13 11:44:02 (348 KB/s) - ‘./anh/capstone_data/crimes.csv’ saved [43289420]



In [60]:
# Read cvs download into datafram
df = pd.read_csv('./anh/capstone_data/crimes.csv',
                 delimiter = ',')

Let's have a quick look at the Head of the imported data

In [61]:
df.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,JD163753,02/24/2020 08:15:00 PM,031XX W LEXINGTON ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,N,N,1134,24.0,11,NaN,NaN,NaN,NaN,NaN
1,JD212847,04/10/2020 10:56:00 PM,005XX W 103RD ST,0560,ASSAULT,SIMPLE,RESIDENCE,N,N,2232,9.0,08A,1174583.0,1836593.0,41.707001,-87.636288,"(41.707000821, -87.636288063)"
2,JD283431,07/01/2020 12:00:00 AM,078XX S MARYLAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,Y,624,8.0,08B,NaN,NaN,NaN,NaN,NaN
3,JC497784,11/03/2019 11:40:00 AM,032XX N CLARK ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,N,N,1924,44.0,06,NaN,NaN,NaN,NaN,NaN
4,JD292044,07/04/2020 08:23:00 PM,028XX W 66TH ST,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,N,N,831,17.0,26,NaN,NaN,NaN,NaN,NaN


In [62]:
df.tail()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
231674,JC339338,07/08/2019 01:04:00 AM,003XX S LARAMIE AVE,5002,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,N,N,1522,29.0,26,1141786.0,1897763.0,41.875527,-87.754885,"(41.875527358, -87.754884881)"
231675,JC368418,07/28/2019 05:00:00 AM,017XX N HUMBOLDT BLVD,0810,THEFT,OVER $500,SIDEWALK,N,N,1421,1.0,06,1156105.0,1911399.0,41.912669,-87.701942,"(41.912668957, -87.70194219)"
231676,JC356571,07/19/2019 09:00:00 PM,031XX W HARRISON ST,5002,OTHER OFFENSE,OTHER VEHICLE OFFENSE,GOVERNMENT BUILDING/PROPERTY,N,N,1134,24.0,26,1155457.0,1897193.0,41.873699,-87.704705,"(41.873699424, -87.704705156)"
231677,JC405144,08/23/2019 11:00:00 PM,019XX W SCHILLER ST,0810,THEFT,OVER $500,STREET,N,N,1424,1.0,06,1163350.0,1909452.0,41.907177,-87.675381,"(41.907176814, -87.675380762)"
231678,JC379670,08/05/2019 12:30:00 PM,074XX S HARVARD AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,Y,731,6.0,08B,1175245.0,1855561.0,41.759037,-87.633299,"(41.759036773, -87.633298885)"


---
### Clean up the data and prepare

The sample data file was downloaded in the middle of July. So half of July of 2019 was missing and half of July 2020 was not yet populated. To simply resolve this the raw data was altered to move the July 2019 dates to July 2020. 

Now that the data has been imported it needs to be cleaned.
1. Move July 2019 dates to July 2020
1. Clean up the column names:
    1. Strip leading & trailing whitespace
    1. Replace multiple spaces with a single space
    1. Remove # characters
    1. Replace spaces with _
    2. Convert to lowercase
1. Change the date of occurance field to a date / time object
1. Add new columns for:
    1. Hour
    1. Day
    1. Month
    1. Year
    1. etc.
1. Split Block into zip_code and street
1. Verify that all rows have valid data

In [63]:
# Strip leading & trailing whitespace
df.columns = df.columns.str.strip()

# Replace multiple spaces with a single space
df.columns = df.columns.str.replace('\s{2,}', ' ')

# Replace # with blank
df.columns = df.columns.str.replace('#', '')

# Replace spaces with _
df.columns = df.columns.str.replace(' ', '_')

# Convert to lowercase
df.columns = df.columns.str.lower()

# Move July 2019 dates to July 2020
df.date_of_occurrence.replace(to_replace="(07/\\d+)/2019", value=r"\1/2020", regex=True, inplace=True)

In [64]:
# Verify that all datatype are as expected
df.dtypes

case                      object
date_of_occurrence        object
block                     object
iucr                      object
primary_description       object
secondary_description     object
location_description      object
arrest                    object
domestic                  object
beat                       int64
ward                     float64
fbi_cd                    object
x_coordinate             float64
y_coordinate             float64
latitude                 float64
longitude                float64
location                  object
dtype: object

Change the date of occurance field to a date / time object

In [65]:
df['date_of_occurrence'] =  pd.to_datetime(df['date_of_occurrence'], format='%m/%d/%Y %I:%M:%S %p')

Now that the date date of the crime is a real date time object let's add new columns for the hour, day, month and year of the crime etc:
1. Hour
1. Day Name
1. Day of week (Tuesday is the first day)
1. Month Name
1. Month Number
1. Year
1. Year and Month

In [66]:
# Add new columns to the dataframe to allow hourly, daily & monthly analysis
df['hour'] = df['date_of_occurrence'].dt.hour
df['day_name'] = df['date_of_occurrence'].dt.day_name()
df['day'] = df['date_of_occurrence'].dt.dayofweek + 1
df['month_name'] = df['date_of_occurrence'].dt.month_name()
df['month'] = df['date_of_occurrence'].dt.month
df['year'] = df['date_of_occurrence'].dt.year
df['year_month'] = df['date_of_occurrence'].dt.to_period('M')

The Block attribute can be split into zip_code and street

In [67]:
# Add the zip and street attributes
df['zip'] = df.block.str.split(' ').str[0]
df['street'] = df.block.str.split(' ').str[1:].apply(', '.join)

Verify that all rows have valid data

In [68]:
# Verify that all rows have valid data
df.isna().sum()

case                        0
date_of_occurrence          0
block                       0
iucr                        0
primary_description         0
secondary_description       0
location_description     1024
arrest                      0
domestic                    0
beat                        0
ward                       11
fbi_cd                      0
x_coordinate             1342
y_coordinate             1342
latitude                 1342
longitude                1342
location                 1342
hour                        0
day_name                    0
day                         0
month_name                  0
month                       0
year                        0
year_month                  0
zip                         0
street                      0
dtype: int64

Some Latitude and Longitude values are missing, drop them.

In [69]:
# Drop rows with missing values 
df.dropna(inplace=True)

In [70]:
# Reindex
df.reset_index(inplace=True)

In [71]:
# Have a final look at the crime dataframe
df.head()

,index,case,date_of_occurrence,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,x_coordinate,y_coordinate,latitude,longitude,location,hour,day_name,day,month_name,month,year,year_month,zip,street
0,1,JD212847,2020-04-10 22:56:00,005XX W 103RD ST,0560,ASSAULT,SIMPLE,RESIDENCE,N,N,2232,9.0,08A,1174583.0,1836593.0,41.707001,-87.636288,"(41.707000821, -87.636288063)",22,Friday,5,April,4,2020,2020-04,005XX,"W, 103RD, ST"
1,11,JD226153,2020-05-07 00:50:00,064XX S DR MARTIN LUTHER KING JR DR,0560,ASSAULT,SIMPLE,RESIDENCE,N,N,312,20.0,08A,1180030.0,1862317.0,41.777468,-87.615556,"(41.777467749, -87.615555593)",0,Thursday,4,May,5,2020,2020-05,064XX,"S, DR, MARTIN, LUTHER, KING, JR, DR"
2,12,JD229722,2020-05-03 02:00:00,024XX S ARCHER AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,STREET,N,N,914,11.0,11,1171590.0,1887793.0,41.847566,-87.645749,"(41.847565598, -87.645749253)",2,Sunday,7,May,5,2020,2020-05,024XX,"S, ARCHER, AVE"
3,14,JD229894,2020-04-28 18:00:00,067XX S PAXTON AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,N,N,331,5.0,07,1192034.0,1860809.0,41.773046,-87.571599,"(41.773046314, -87.571598578)",18,Tuesday,2,April,4,2020,2020-04,067XX,"S, PAXTON, AVE"
4,15,JD226735,2020-05-07 17:17:00,043XX W NORTH AVE,0560,ASSAULT,SIMPLE,SMALL RETAIL STORE,N,N,2534,26.0,08A,1146821.0,1910272.0,41.909759,-87.736078,"(41.909758833, -87.736078373)",17,Thursday,4,May,5,2020,2020-05,043XX,"W, NORTH, AVE"


In [73]:
df = df[['index', 'case', 'date_of_occurrence', 'block', 'primary_description', 'ward', 'latitude', 'longitude', 'hour', 'day_name', 'day', 'month_name', 'month', 'year', 'year_month', 'zip', 'street']]
df.shape

(229405, 17)

In [74]:
df.head(10)

,index,case,date_of_occurrence,block,primary_description,ward,latitude,longitude,hour,day_name,day,month_name,month,year,year_month,zip,street
0,1,JD212847,2020-04-10 22:56:00,005XX W 103RD ST,ASSAULT,9.0,41.707001,-87.636288,22,Friday,5,April,4,2020,2020-04,005XX,"W, 103RD, ST"
1,11,JD226153,2020-05-07 00:50:00,064XX S DR MARTIN LUTHER KING JR DR,ASSAULT,20.0,41.777468,-87.615556,0,Thursday,4,May,5,2020,2020-05,064XX,"S, DR, MARTIN, LUTHER, KING, JR, DR"
2,12,JD229722,2020-05-03 02:00:00,024XX S ARCHER AVE,DECEPTIVE PRACTICE,11.0,41.847566,-87.645749,2,Sunday,7,May,5,2020,2020-05,024XX,"S, ARCHER, AVE"
3,14,JD229894,2020-04-28 18:00:00,067XX S PAXTON AVE,MOTOR VEHICLE THEFT,5.0,41.773046,-87.571599,18,Tuesday,2,April,4,2020,2020-04,067XX,"S, PAXTON, AVE"
4,15,JD226735,2020-05-07 17:17:00,043XX W NORTH AVE,ASSAULT,26.0,41.909759,-87.736078,17,Thursday,4,May,5,2020,2020-05,043XX,"W, NORTH, AVE"
5,16,JD229788,2020-04-25 15:03:00,007XX N SACRAMENTO BLVD,DECEPTIVE PRACTICE,27.0,41.893990,-87.702156,15,Saturday,6,April,4,2020,2020-04,007XX,"N, SACRAMENTO, BLVD"
6,17,JD226263,2020-05-07 08:15:00,008XX N LATROBE AVE,THEFT,37.0,41.895581,-87.756842,8,Thursday,4,May,5,2020,2020-05,008XX,"N, LATROBE, AVE"
7,18,JD226234,2020-05-07 06:00:00,012XX E MADISON AVENUE PARK,BATTERY,4.0,41.802916,-87.596079,6,Thursday,4,May,5,2020,2020-05,012XX,"E, MADISON, AVENUE, PARK"
8,19,JD229900,2020-05-05 00:00:00,023XX E 70TH PL,CRIMINAL DAMAGE,5.0,41.767135,-87.568751,0,Tuesday,2,May,5,2020,2020-05,023XX,"E, 70TH, PL"
9,20,JD229801,2020-04-26 12:00:00,073XX N DAMEN AVE,THEFT,49.0,42.014719,-87.680489,12,Sunday,7,April,4,2020,2020-04,073XX,"N, DAMEN, AVE"


In [75]:
# Finally write the dataframe to a pickle file for restoring later
df.to_pickle('./anh/capstone_pickles/crimes.pkl')

In [5]:
df[['primary_description', 'case']].groupby(
    ['primary_description'], as_index=False).count().sort_values(
    'case', ascending=False).head(3)

,primary_description,case
30,THEFT,52465
2,BATTERY,45332
6,CRIMINAL DAMAGE,25651


The data is now ready for visualisation.